In [ ]:
!pip3 install python-chess

In [8]:
import pandas as pd
import ast
from collections import defaultdict as dd
import regex as re
import seaborn as sns
import numpy as np
import chess
from itertools import chain


# Descriptive Analytics

In [9]:
df = pd.read_csv("sample.csv", header=0, index_col = 0)

/home/isaac/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df.head()

,Event,Site,GameNumber,WName,BName,WElo,BElo,WRD,BRD,WIsComp,...,TimeControl,Date,Time,WClock,BClock,ECO,PlyCount,Result,WMoves,BMoves
0,FICS rated blitz game,FICS freechess.org,470452425,chesspickle,lawrencegern,1963,764,21.2,112.2,Yes,...,300+0,2020.01.31,23:54:00,05:00.0,05:00.0,D11,43,1-0,"['d4', 'c4', 'Nf3', 'e3', 'Bxc4', 'Be2', 'Bd2'...","['d5', 'c6', 'dxc4', 'e6', 'b5', 'Bb4+', 'Bxd2..."
1,FICS rated blitz game,FICS freechess.org,470452424,ZoneBlindTal,AhNeow,1600,1487,36.6,28.5,NaN,...,180+2,2020.01.31,23:52:00,03:00.0,03:00.0,C41,48,0-1,"['e4', 'Nf3', 'd4', 'dxe5', 'Nc3', 'Nd5', 'Bd2...","['e5', 'd6', 'Qe7', 'dxe5', 'f6', 'Qd6', 'c6',..."
2,FICS rated blitz game,FICS freechess.org,470452423,forlat,naraindra,1948,1089,15.3,30.7,Yes,...,300+0,2020.01.31,23:54:00,05:00.0,05:00.0,D00,67,1-0,"['d4', 'g3', 'Bg2', 'Nf3', 'O-O', 'e3', 'c3', ...","['d5', 'b5', 'Bb7', 'h6', 'e6', 'Bd6', 'Nf6', ..."
3,FICS rated blitz game,FICS freechess.org,470452420,PGierech,kkpsA,1497,1224,32.4,37.6,NaN,...,120+12,2020.01.31,23:55:00,02:00.0,02:00.0,D00,23,1-0,"['e4', 'd4', 'dxe5', 'exd5', 'Nf3', 'Be2', 'Bx...","['d5', 'e5', 'Nc6', 'Nxe5', 'Bg4', 'Nxf3+', 'B..."
4,FICS rated blitz game,FICS freechess.org,470452419,Liszt,Sotavaunu,1594,1554,26.8,34.9,NaN,...,300+2,2020.01.31,23:48:00,05:00.0,05:00.0,B06,53,0-1,"['e4', 'd4', 'f4', 'c3', 'cxd4', 'e5', 'Nc3', ...","['g6', 'Bg7', 'c5', 'cxd4', 'd5', 'Nc6', 'Nh6'..."


In [11]:
df["BMoves"] = df["BMoves"].apply(ast.literal_eval)
df["WMoves"] = df["WMoves"].apply(ast.literal_eval)

In [36]:
df = df[df["WMoves"] != "eow"]
df = df[df["BMoves"] != "eow"]

In [ ]:
df.loc[2,'WMoves']

## Most popular square?

In [14]:
lookups = [
    ("^.*[a-h][1-8].*$", "normal move"),
    ("^O-O$", "kingside castle"),
    ("^O-O-O$", "queenside castle"),
    ("^.+[QNBR]\+*#*$", "pawn conversion"),
    ("^.*\+$", "check"),
    ("^.*#$", "checkmate")
]

def is_not_castle(move):
    return not bool(re.search("O-O",move))


In [ ]:
square_dict = dd(int)

def count_squares(moves, colour):
    for move in moves:
        if is_not_castle(move):
            square_dict[re.findall("[a-h][1-8]",move)[-1]] += 1
        else:
            if re.search("^O-O$", move):
                if colour=="W":
                    square_dict["g1"] += 1
                else:
                    square_dict["g8"] += 1
            else: 
                if colour=="W":
                    square_dict["c1"] += 1
                else:
                    square_dict["c8"] += 1
    return


In [ ]:
df["BMoves"].apply(lambda x : count_squares(x,'B'))
df["WMoves"].apply(lambda x : count_squares(x,'W'))

## How many moves until a game becomes unique?

In [ ]:
df.loc[195]

In [ ]:
sample_df = df.head(1000)

In [ ]:
moves_until_unique = dd(int)

for i in range(len(sample_df["WMoves"])):
    for j in range(i,len(sample_df["WMoves"])):
        move_count = -1
        g1move = ""
        g2move = ""
        while g1move==g2move:
            move_count += 1
            try:
                if move_count%2 == 0:
                    #White's move
                    g1move = df["WMoves"][i][move_count//2]
                    g2move = df["WMoves"][j][move_count//2]
                else:
                    #Black's move
                    g1move = df["BMoves"][i][move_count//2]
                    g2move = df["BMoves"][j][move_count//2]
            except:
                #end of game
                g1move = 1
                g2move = 2
        if moves_until_unique[i] < move_count:
            moves_until_unique[i] = move_count
        if moves_until_unique[j] < move_count:
            moves_until_unique[j] = move_count
            

In [ ]:
moves_until_unique

## CapturesByPiece
function takes as argument the pandas dataframe and returns the pandas dataframe with capture count for pair of pieces

In [ ]:
def capturesByPiece(df):
    # Dict of dict to store the count of captures made by a piece

    captureCount = dict()
    for attacker in chess.UNICODE_PIECE_SYMBOLS.keys():
        captureCount[attacker] = dict()
        for captured in chess.UNICODE_PIECE_SYMBOLS.keys():
            captureCount[attacker][captured] = 0
    
  
            
    for index, row in df.iterrows():
        WMoves = row["WMoves"]
        BMoves = row["BMoves"]

        board = chess.Board()

        # zipping all the moves and merging into one list 
        allMoves = zip(WMoves, BMoves)
        zippedMoves = list(chain.from_iterable(allMoves))

        for move in zippedMoves:
            if board.is_capture(board.parse_san(move)):
                moveInUCI = board.uci(board.parse_san(move))
#                 print("move:",move)
                attackerPosition, capturedPosition = moveInUCI[:2], moveInUCI[2:4]
#                 print("ap,cp:"+attackerPosition, capturedPosition)
                attacker = board.piece_at(chess.SQUARE_NAMES.index(attackerPosition))
                captured = board.piece_at(chess.SQUARE_NAMES.index(capturedPosition))
#                 print("a,c:",attacker,captured)
                
                if (str(attacker) in captureCount.keys()) and (str(captured) in captureCount.keys()):
                    captureCount[str(attacker)][str(captured)] += 1
        
            board.push_san(move)
        
#         print()
    
    capture_count = pd.DataFrame(captureCount)
    capture_count.rename(columns = chess.UNICODE_PIECE_SYMBOLS, index =  chess.UNICODE_PIECE_SYMBOLS, inplace = True) 

    return  capture_count

In [ ]:
%%time
sns.heatmap(capturesByPiece(df.head(5)), cmap="YlGnBu")

# Prescriptive Analytics

## Feature Engineering

## Add columns for who won game

In [15]:
def get_player_result(result, colour):
    if colour=='B':
        if result[-1]=='0':
            return 0
        else:
            return 1
    else:
        if result[0]=='0':
            return 0
        else:
            return 1

def add_WL_columns(df):
    df['WResult'] = df["Result"].apply(lambda x: get_player_result(x, 'W'))
    df['BResult'] = df["Result"].apply(lambda x: get_player_result(x, 'B'))
    return

In [16]:
add_WL_columns(df)

## Add columns for pawn conversions

In [17]:
def get_conversions(moves):
    conversions = []
    for move in moves:
        if re.match(lookups[3][0], move) is not None:
            conversions.append(re.search("(?<=[a-h][1-8]=)[QNBR]",move).group(0))
    return len(conversions)

def add_conversions_columns(df):
    df["WConversions"] = df["WMoves"].apply(lambda x: get_conversions(x))
    df["BConversions"] = df["BMoves"].apply(lambda x: get_conversions(x))

In [18]:
add_conversions_columns(df)

**Add columns for what piece was used the most by a player**

In [19]:
df['WFavPiece'] = 'null'
df['BFavPiece'] = 'null'

WPieceMoves = {'N': 0, 'B': 0, 'R': 0, 'Q': 0, 'K':0}
BPieceMoves = {'N': 0, 'B': 0, 'R': 0, 'Q': 0, 'K':0}

def get_favPiece():
    for i in range(len(df)):
        moves = df.loc[i, 'WMoves']
        for move in moves:
            if(move):
                if move[0].isupper() and move[0] != 'O':
                    WPieceMoves[move[0]] += 1
        FavPiece = max(WPieceMoves, key=WPieceMoves.get)
        df.loc[i, ['WFavPiece']] = FavPiece
        
        moves = df.loc[i, 'BMoves']
        for move in moves:
            if(move):
                if move[0].isupper() and move[0] != 'O':
                    BPieceMoves[move[0]] += 1
        FavPiece = max(BPieceMoves, key=BPieceMoves.get)
        df.loc[i, ['BFavPiece']] = FavPiece

In [20]:
get_favPiece()

## Moves Count
Moves can either be ordinary move or capturing move, 
 - `WCaptureMovesCount` represent number of capturing moves in the game made by White player
 - `BCaptureMovesCount` represent number of capturing moves in the game made by Black player
 - `WOrdMovesCount` represent number of ordinary moves in the game made by White player
 - `BOrdMovesCount` represent number of oridnary moves in the game made by Black player

In [21]:
def captureCount(moves):
    count = 0
    for move in moves:
        if "x" in move:
            count += 1
    return count
            

In [22]:
def ordinaryCount(moves):
    count = 0
    for move in moves:
        if "x" not in move:
            count += 1
    return count

In [23]:
def addmovesCount():
    df["WCaptureMovesCount"] = df["WMoves"].apply(captureCount)
    df["BCaptureMovesCount"] = df["BMoves"].apply(captureCount)

    df["WOrdMovesCount"] = df["WMoves"].apply(ordinaryCount)

    df["BOrdMovesCount"] = df["BMoves"].apply(ordinaryCount)

In [24]:
addmovesCount()

## Number of pieces at the end of the game

In [25]:
def addpieces():
    df["WPiecesCount"] = np.abs(df["BCaptureMovesCount"].subtract(16))

    df["BPiecesCount"] = np.abs(df["WCaptureMovesCount"].subtract(16))

In [26]:
addpieces()

## Pairwise capture count
    - Uppercase letter(R,N,B,Q,K,P) represent White pieces 
    - lowercase letter(r,n,b,q,k,p) represent Black pieces
    - Column "Pp" represent P captures p count

In [ ]:
def pairCaptureDict(WMoves, BMoves):
#     Dictionary to store count
    captureCount = dict()
    for attacker in chess.UNICODE_PIECE_SYMBOLS.keys():
        for captured in chess.UNICODE_PIECE_SYMBOLS.keys():
            key = attacker+captured
            if (not key.isupper()) and (not key.islower()):
                captureCount[key] = 0
                
                
    board = chess.Board()
 # zipping all the moves and merging into one list 
    allMoves = zip(WMoves, BMoves)
    zippedMoves = list(chain.from_iterable(allMoves))

    for move in zippedMoves:
        if board.is_capture(board.parse_san(move)):
            moveInUCI = board.uci(board.parse_san(move))
#                 print("move:",move)
            attackerPosition, capturedPosition = moveInUCI[:2], moveInUCI[2:4]
#                 print("ap,cp:"+attackerPosition, capturedPosition)
            attacker = board.piece_at(chess.SQUARE_NAMES.index(attackerPosition))
            captured = board.piece_at(chess.SQUARE_NAMES.index(capturedPosition))
#                 print("a,c:",attacker,captured)
            key = (str(attacker)+str(captured))
            if (key in captureCount.keys()):
                    captureCount[key] += 1
        board.push_san(move)
    return captureCount


In [ ]:
#  Adds the features to the dataframe 
df['pairCaptureDict'] = df.apply(lambda x: pairCaptureDict(x.WMoves, x.BMoves), axis=1)

new_df = pd.DataFrame(list(df["pairCaptureDict"]))

df = pd.concat([df, new_df], axis=1)

df = df.drop(columns = "pairCaptureDict")

In [ ]:
df.head()

## finding first check

In [2]:
def find_check(entry):
    n_moves = 0;
    for i in range(len(entry["WMoves"])):
        n_moves+=1
        if entry["WMoves"][i][-1] == "+":
            return n_moves
        if i == len(entry["WMoves"]) - 1 and len(entry["WMoves"]) != len(entry["BMoves"]):
            return -1
        if entry["BMoves"][i][-1] == "+":
            return n_moves
    return -1

def find_checks(df):
    chks = []
    for i in range(len(df)):
        chks.append(find_check(df.iloc[i]))
    df["first_check"] = chks

In [27]:
def find_first_check(moves):
    for i in range(len(moves)):
        if moves[i][-1] == "+":
            return i
    return len(moves)

def get_check_columns(df):
    df["WCheck"] = df["WMoves"].apply(lambda x : find_first_check(x))
    df["BCheck"] = df["BMoves"].apply(lambda x : find_first_check(x))

In [28]:
get_check_columns(df)

In [ ]:
pd.set_option('display.max_columns', None)

## High skill openings

In [31]:
def high_skill_openings(df):
    open_elo = dd(int)
    open_count = dd(int)
    bOpens = []
    wOpens = []
    for i in range(len(df)):
        wOpen = tuple(df["WMoves"][i][0:5])
        wOpens.append(wOpen)
        wSkill = df["WElo"][i]
        open_elo[wOpen] += int(wSkill)
        open_count[wOpen] += 1
        bOpen = tuple(df["BMoves"][i][0:5])
        bOpens.append(wOpen)
        bSkill = df["BElo"][i]
        open_elo[bOpen] += bSkill
        open_count[bOpen] += 1
        
    for key in open_elo.keys():
        open_elo[key] = open_elo[key]/open_count[key]
    return open_elo, bOpens, wOpens

def openings(df):
    elos, bOpens, wOpens = high_skill_openings(df)
    df["bOpens"] = bOpens
    df["wOpens"] = wOpens

In [37]:
openings(df)

ValueError: invalid literal for int() with base 10: 'eow'

In [ ]:
df.head()

**Score Calculation**

In [38]:
InitialBoardW = ['Pa2', 'Pb2', 'Pc2', 'Pd2', 'Pe2', 'Pf2', 'Pg2', 'Ph2', 'Ra1', 'Nb1', 'Bc1', 'Qd1', 'Ke1', 'Bf1', 'Ng1', 'Rh1']
InitialBoardB = ['Pa7', 'Pb7', 'Pc7', 'Pd7', 'Pe7', 'Pf7', 'Pg7', 'Ph7', 'Ra8', 'Nb8', 'Bc8', 'Qd8', 'Ke8', 'Bf8', 'Ng8', 'Rh8']

PieceVal = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K':100}

def calculateScore(movesList):
    scoreW = scoreB = 39                                                                                                                           
    for i in range(len(movesList)):
        if 'x' in movesList[i]:       
            posIndex = movesList[i].index('x') + 1
            position = movesList[i][posIndex:]
            j = i-1     
            while j >= 0:
                if position in movesList[j]:
                    if position[0] == movesList[j][0]:
                        if j%2 == 1:
                            scoreB -= 1
                        else:
                            scoreW -= 1
                    elif 'x' in movesList[j]:
                        if movesList[j][0].islower():
                            if j%2 == 1:
                                scoreB -= 1
                            else:
                                scoreW -= 1
                        else:
                            if j%2 == 1:
                                scoreB -= PieceVal[movesList[j][0]]
                            else:
                                scoreW -= PieceVal[movesList[j][0]]
                            
                    else:
                        if j%2 == 1:
                            scoreB -= PieceVal[movesList[j][0]]
                        else:
                            scoreW -= PieceVal[movesList[j][0]]
                j -= 1
                
            if i%2 == 0:
                for piece in InitialBoardB:
                    if position in piece:
                        scoreB -= PieceVal[piece[0]]
            else:
                for piece in InitialBoardW:
                    if position in piece:
                        scoreW -= PieceVal[piece[0]]        
        
    return scoreW, scoreB

In [39]:
completeList = []
for i in range(len(df)):
    moves = []
    wmoves = df.loc[i,'WMoves']
    bmoves = df.loc[i,'BMoves']
    if len(wmoves) > len(bmoves):
        for k in range(len(bmoves)):
            moves.append(wmoves[k])
            moves.append(bmoves[k])
        #moves.append(wmoves[k+1])
    elif len(wmoves) == len(bmoves):
        for k in range(len(bmoves)):
            moves.append(wmoves[k])
            moves.append(bmoves[k])
    else:
        for k in range(len(wmoves)):
            moves.append(wmoves[k])
            moves.append(bmoves[k])
        moves.append(bmoves[k+1])
    completeList.append(moves)
df['Moves'] = completeList

In [40]:
wscoreList = []
bscoreList = []
for i in range(len(df)):
    w, b = calculateScore(df.loc[i, 'Moves'])
    wscoreList.append(w)
    bscoreList.append(b)
    
df['WScore'] = wscoreList
df['BScore'] = bscoreList

# Model Building

In [59]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RANSACRegressor, Perceptron, RidgeCV, SGDRegressor, ElasticNetCV, LarsCV, LassoCV, LassoLarsCV, OrthogonalMatchingPursuitCV, BayesianRidge, ARDRegression, MultiTaskElasticNetCV, PoissonRegressor, TweedieRegressor, GammaRegressor, enet_path
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
BX = df[["BResult", "BConversions", "BCaptureMovesCount", "BOrdMovesCount","BPiecesCount","BCheck","WCheck","WScore","BScore"]]
By = df["BElo"]

In [60]:
train_bx, test_bx, train_by, test_by= train_test_split(BX, By)
models = [MLPRegressor, RidgeCV, SGDRegressor, ElasticNetCV, LarsCV, LassoCV, LassoLarsCV, OrthogonalMatchingPursuitCV, BayesianRidge, RANSACRegressor, AdaBoostRegressor, ARDRegression, PoissonRegressor, TweedieRegressor, GammaRegressor]
for model in models:
    cur_model = model()
    cur_model.fit(train_bx, train_by)
    by_pred = cur_model.predict(test_bx)
    print("{}: r2:{} MSE:{}".format(type(cur_model).__name__, r2_score(by_pred, test_by),mean_squared_error(by_pred, test_by)))
type(model).__name__

/home/isaac/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor: r2:-10.91433683929304 MSE:50117.84789229372
RidgeCV: r2:-19.380911478578522 MSE:49769.17529163217
SGDRegressor: r2:-0.05624553234036078 MSE:1.2939728733838546e+22
ElasticNetCV: r2:-35.30066684834658 MSE:50438.95108744587
LarsCV: r2:-19.51797162651046 MSE:49770.30366600192
LassoCV: r2:-20.497529976051972 MSE:49779.696028915256
LassoLarsCV: r2:-19.51797162651046 MSE:49770.30366600192
OrthogonalMatchingPursuitCV: r2:-19.74348828279471 MSE:49826.03975737427
BayesianRidge: r2:-19.620005316112255 MSE:49771.50412071645
RANSACRegressor: r2:-2.7019611578910494 MSE:88342.86650542244
AdaBoostRegressor: r2:-6.748636697905253 MSE:54142.96484674032
ARDRegression: r2:-19.557683051500998 MSE:49768.0718968903
PoissonRegressor: r2:-2.5513231544590714e+29 MSE:52760.12624315444
TweedieRegressor: r2:-35.70891235886043 MSE:50457.8215660092
GammaRegressor: r2:-2.5513231544590714e+29 MSE:52760.12624315444


/home/isaac/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/link.py:90: RuntimeWarning: overflow encountered in exp
  return np.exp(lin_pred)
/home/isaac/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/link.py:93: RuntimeWarning: overflow encountered in exp
  return np.exp(lin_pred)
/home/isaac/.local/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:132: RuntimeWarning: invalid value encountered in true_divide
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
/home/isaac/.local/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:315: RuntimeWarning: invalid value encountered in add
  dev = 2 * (xlogy(y, y/y_pred) - y + y_pred)
/home/isaac/.local/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:132: RuntimeWarning: overflow encountered in multiply
  return -2 * (y - y_pred) / self.unit_variance(y_pred)
/home/isaac/.local/lib/python3.8/site-packages/sklearn/_loss/glm_distribution.py:315: RuntimeWarning: overflow encounte

'type'

In [61]:
bmean = df["BElo"].mean()
bmeans = [bmean for i in range(len(by_pred))]

mean_squared_error(bmeans,test_by)

52757.471111292485

In [ ]:
model = 